- 참고
    - 블로그 : https://pkgpl.org/2023/11/08/openai-assistants-api로-대화-내용-저장하기/
    - 레퍼런스 : https://platform.openai.com/docs/assistants/overview

- openai assistents api가 langchain과 다른점 : memory가 아니라 thread를 이용해 대화를 저장한다.
    - 장점 : 토큰 제한을 알아서 맞춰준다.
    - 단점 : 커스터마이징할 수 있는 부분이 제한적이고, 대화 내용을 최대한 보존하는 방식으로 토큰 소모가 많다.

- 용어 정리
    - **Assistant**: 도구들을 사용할 수 있는 **거대 언어 모델**(GPT 모델들)입니다.
    - **`Thread`** : **`Message들을 저장하는 대화`** 입니다.
    - **Message**: 대화에서 주고 받는 메시지들로, 텍스트 외에도 **이미지나 파일을 포함**할 수 있습니다.
    - **`Run`**: Assistant가 Thread에 저장된 Message들을 읽고 **`LLM의 응답 Message를 추가하는 과정`** 입니다.
    - **Run Step**: Run을 실행할 때 단순히 LLM의 응답을 추가하는 경우도 있지만, 외부 도구들(Tools)을 사용한 후 결과를 이용해 응답하는 경우도 있습니다. **어떤 세부 단계를 거쳐 응답했는지 확인하고 싶을 때** Run Step을 이용합니다.

- Run상태
    - **기본** 응답: **queued → in_progress → completed**
    - Function calling **`(사용자가 제공한 Tool) 사용`** 시: queued → in_progress → **`requires_action → (함수 호출 결과 입력 후 다시 Run) → queued → in_progress`** → completed
    - 응답 시간이 **오래** 걸리면: queued → in_progress → **expired**
    - 응답 **실패**시: queued → in_progress → **failed**
    - **사용자가 취소**할 경우: queued → in_progress → **(사용자가 취소) → cancelling → cancelled**

In [8]:
pwd

'/home/t16user5/gpt4'

In [2]:
import dotenv
from dotenv import load_dotenv
import openai
import os
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# assert version is > 1.0.0
assert openai.__version__ > "1.0.0", "Please upgrade the OpenAI Python client: pip install -U openai"
print(openai.__version__)

1.3.7


In [4]:
pip install -U openai

Note: you may need to restart the kernel to use updated packages.


# csv 파일 업로드 
## Files for context of the assistant

In [45]:
import glob
# data 폴더 내부 및 하위 폴더에서 모든 Python 파일 가져오기
filepath = "data/"
python_files = glob.glob(filepath + "**/*.csv", recursive=True)
print(f"Found {len(python_files)} python files")

def is_usefull_python_file(python_file_path):
    # Assistant에 대한 불필요한 파일 제거를 위한 함수
    bad_strings_to_not_match = ["test", "mlperf", "setup.py",
                                "runtime", "examples", "renderer",
                                 "openpilot", "models","adreno", 
                                 "extra", "jit", "sz.py"]
    if "resnet.py" in python_file_path:
        return True
    for bad_string in bad_strings_to_not_match:
        if bad_string in python_file_path:
            return False
    return True

# 파일 이름에 "model"이 포함된 파일 출력
print("Files with model in the path")
print(list(filter(lambda x: "model" in x, python_files)))

# 유용한 Python 파일만 필터링 후 출력
python_files = list(filter(is_usefull_python_file, python_files))
print(f"Found {len(python_files)} python files after filtering")
assert len(python_files) <= 20, "Openai limit is 20 files"
print("\n".join(python_files))

Found 1 python files
Files with model in the path
[]
Found 1 python files after filtering
data/qa1-simp.csv


## Upload the files to Openai

In [46]:
from openai import OpenAI
from tqdm.auto import tqdm
client = OpenAI()
uploaded_files = []
# tqdm을 사용하여 파일 업로드 진행 상황 표시
for filename in tqdm(python_files):
    file_obj = client.files.create(
        file=open(filename, 'rb'),
        purpose='assistants',
        
    )
    uploaded_files.append(file_obj.id)

  0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
uploaded_files

['file-xsaPm7iZhUDzz9QFhhHXEEnB']

## (Assistant) LLM 만들기
- Openai의 서버에 저장, 고유한 id보유

In [55]:
tinygrad_assistant = openai.beta.assistants.create(
    instructions="You are helping coders understand the tinygrad library. You have access to the source code of the codebase. You are a helpful tinygrad assistant.",
    name="Tinygrad Assistant (API)",
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=uploaded_files,
)

## Thread 만들기 (Create an empty thread)
Empty threads are not linked to an assistant yet. Multiple assistants can be used inside the same thread :)

In [56]:
# 빈 thread 생성
thread = openai.beta.threads.create()
thread_id = thread.id

# 메시지 생성
query = "Q가 말한 것은 몇가지야?"
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content=query,
)

# run 상태확인
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=tinygrad_assistant.id)

# 응답 소요시간 및 완성 체크
import time
while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread_id, 
        run_id=run.id)
    if run.completed_at:
        elapsed = run.completed_at - run.created_at
        elapsed = time.strftime("%H:%M:%S", time.gmtime(elapsed))
        print(f"Run completed in {elapsed}")
        break
    print("Waiting 1sec")
    time.sleep(1)

Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Run completed in 00:00:11


## Thread에 추가된 응답 확인

In [58]:
messages = client.beta.threads.messages.list(thread_id=thread_id)
last_message = messages.data[0]

text = last_message.content[0].text.value
print(text)

Q가 말한 것은 총 7가지입니다. 다음과 같습니다:

1. 일이 잘못되면 스스로를 낮추거나 의심하거나 비난한다.
2. 일이 잘 풀리지 않을 거라 생각한다.
3. 다른 사람들보다 능력이 부족하다고 믿어 자신의 능력보다 적게 성취한다.
4. 비판, 거절에 화를 내거나 괴로워한다.
5. 다른 사람을 기쁘게 하기 위해 상대에게 맞추거나 자신을 깎아내린다.
6. 여가 활동을 많이 하지 않는다.
7. 자기 관리에 영향을 받는다.

이는 낮은 자존감으로 인해 나타나는 영향들을 나열한 것입니다.


In [59]:
for msg in reversed(messages.data):
    if msg.role=='user':
        print("################질문################")
    else:
        print("################응답################")
    print(f"{msg.role}: {msg.content[0].text.value}")

################질문################
user: Q가 말한 것은 몇가지야?
################응답################
assistant: Q가 말한 것은 총 7가지입니다. 다음과 같습니다:

1. 일이 잘못되면 스스로를 낮추거나 의심하거나 비난한다.
2. 일이 잘 풀리지 않을 거라 생각한다.
3. 다른 사람들보다 능력이 부족하다고 믿어 자신의 능력보다 적게 성취한다.
4. 비판, 거절에 화를 내거나 괴로워한다.
5. 다른 사람을 기쁘게 하기 위해 상대에게 맞추거나 자신을 깎아내린다.
6. 여가 활동을 많이 하지 않는다.
7. 자기 관리에 영향을 받는다.

이는 낮은 자존감으로 인해 나타나는 영향들을 나열한 것입니다.


## 새로운 메시지 추가

In [60]:
# 메시지 생성
query = "Q을 위해 어떤 그림, 음악, 영상 컨텐츠가 위로가 될까?"
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content=query,
)

# run 상태확인
## run 생성
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=tinygrad_assistant.id)

## 응답 소요시간 및 상태 체크
import time
while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread_id, 
        run_id=run.id)
    if run.completed_at:
        elapsed = run.completed_at - run.created_at
        elapsed = time.strftime("%H:%M:%S", time.gmtime(elapsed))
        print(f"Run completed in {elapsed}")
        break
    print("Waiting 1sec")
    time.sleep(1)

Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Run completed in 00:00:21


In [61]:
messages = client.beta.threads.messages.list(thread_id=thread_id)
for msg in reversed(messages.data):
    if msg.role=='user':
        print("\n ################질문################")
    else:
        print("################응답################")
    print(f"{msg.role}: {msg.content[0].text.value}")


 ################질문################
user: Q가 말한 것은 몇가지야?
################응답################
assistant: Q가 말한 것은 총 7가지입니다. 다음과 같습니다:

1. 일이 잘못되면 스스로를 낮추거나 의심하거나 비난한다.
2. 일이 잘 풀리지 않을 거라 생각한다.
3. 다른 사람들보다 능력이 부족하다고 믿어 자신의 능력보다 적게 성취한다.
4. 비판, 거절에 화를 내거나 괴로워한다.
5. 다른 사람을 기쁘게 하기 위해 상대에게 맞추거나 자신을 깎아내린다.
6. 여가 활동을 많이 하지 않는다.
7. 자기 관리에 영향을 받는다.

이는 낮은 자존감으로 인해 나타나는 영향들을 나열한 것입니다.

 ################질문################
user: Q을 위해 어떤 그림, 음악, 영상 컨텐츠가 위로가 될까?
################응답################
assistant: 파일 내에서 "Q을 위해 어떤 그림, 음악, 영상 컨텐츠가 위로가 될까?"에 대한 구체적인 정보는 찾을 수 없습니다. 파일이 낮은 자존감과 관련된 내용을 다루고 있으므로, Q에게 위로가 될만한 컨텐츠를 제안하기 위해서는 일반적인 지식을 바탕으로 답변드릴 수 있습니다. 

낮은 자존감을 가진 사람에게는 자기 가치를 높이고 긍정적인 기분을 유발할 수 있는 그림, 음악 또는 영상 컨텐츠가 효과적일 수 있습니다. 예를 들면:

- 그림: 자연 풍경이나 동물, 혹은 온기를 느낄 수 있는 가족이나 친구들의 그림이 긍정적인 기분을 유료할 수 있습니다.
- 음악: 가사가 긍정적이고 리듬이 상쾌한 음악은 기분을 개선할 수 있습니다.
- 영상: 동기부여가 되는 다큐멘터리, 유머가 있는 영화나 TV 프로그램, 자연의 아름다움을 보여주는 영상 등이 자존감을 높일 수 있는 컨텐츠입니다.

이런 컨텐츠들은 Q가 스스로를 더 긍정적으로 바라보고 자신을 소중히 여기며 삶의 즐거움을 찾는데 도움을 줄 수 있습니다.


## assistant와 thread 삭제

In [ ]:
# delete thread
response = client.beta.threads.delete(thread.id)
print(response)

# delete assistant
response = client.beta.assistants.delete(tinygrad_assistant.id)
print(response)